In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from PIL import Image
import os
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tensorflow.keras.layers import Conv2D, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
import torch.optim as optim
from tensorflow.keras.optimizers import Adam

In [ ]:
df = pd.read_csv('/kaggle/input/uwmgi-mask-dataset/train.csv')

In [ ]:
unnecessary = ["id", "class", "segmentation", "case", "day", "slice", "height", "width"]

for col in unnecessary:
    df = df.drop([col], axis=1)
    
df.head()

In [ ]:
x=0

good_rows = []

for index, row in df.iterrows():
    if x % 6 == 0:
        good_rows.append(index)
    x += 1

df = df.loc[good_rows]
    
df.shape

In [ ]:
df.head()

In [ ]:
df.to_csv('all_data.csv', index=False)

In [ ]:
def load_and_resize(image_path, mask_path, size = (128, 128)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    image = cv2.resize(image, dsize=size)
    mask = cv2.resize(mask, dsize=size)
    return image, mask

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def augment(image, mask, batch_size=5):
    gen_args = dict(
        horizontal_flip = True,
        vertical_flip = True,
        rotation_range = 90,
        zoom_range = 0.2,
    )
    
    datagen = ImageDataGenerator(**gen_args)
    
    seed = np.random.randint(1, 1000)
    
    if len(image.shape) == 2:
        image = np.expand_dims(image, axis=0)
        image = np.expand_dims(image, axis=3)
    if len(mask.shape) == 2:
        mask = np.expand_dims(mask, axis=0)
        mask = np.expand_dims(mask, axis=3)
    
    # Combine image and mask
    combined = np.concatenate((image, mask), axis=3)
    
    datagen.fit(combined, augment=True, seed=np.random.randint(1, 1000))
    
    combined_gen = datagen.flow(combined, batch_size=1, seed=np.random.randint(1, 1000))

    size = (128, 128)
    
    image = np.squeeze(image)
    mask = np.squeeze(mask)
    
    image = cv2.resize(image, dsize=size)
    mask = cv2.resize(mask, dsize=size)
    
    aug_imgs = [image]
    aug_msks = [mask]

    
    for _ in range(batch_size):
        combined_aug = next(combined_gen)[0]
        
        cur_img = combined_aug[..., :1]
        cur_msk = combined_aug[..., 1:]
        
        cur_img = np.squeeze(cur_img)
        cur_msk = np.squeeze(cur_msk)

        cur_img = cv2.resize(cur_img, dsize=size)
        cur_msk = cv2.resize(cur_msk, dsize=size)

        aug_imgs.append(cur_img)
        aug_msks.append(cur_msk)
    
    return aug_imgs, aug_msks

In [ ]:
def normalize(img):
    img = img.astype(np.float32)
    #normalized_image = cv2.normalize(img, None, alpha=0, beta=1.0, norm_type=cv2.NORM_MINMAX)
    normalized_image = img / 255.0
    return normalized_image

In [ ]:
def pipeline(row):
    img, mask = load_and_resize(df['image_path'][row], df['mask_path'][row])
    img_list, mask_list = augment(img, mask)
    
    #imgplot = plt.imshow(img_list[0])
    #plt.show()
    #print(img_list[0][64][80])
    
    img_list = [normalize(img) for img in img_list]
    mask_list = [normalize(msk) for msk in mask_list]
    
    #imgplot = plt.imshow(img_list[0])
    #plt.show()
    #print(img_list[0][64][80])
    
    return img_list, mask_list

In [ ]:
temp_img = []
temp_msk = []

#x=0
for row in df.index:
    #if (x < 260):
        #x += 1
        #continue
    #if x == 262:
        #break
    
    
    img_list, msk_list = pipeline(row)
    
    #imgplot = plt.imshow(msk_list[0])
    #plt.show()
    
    for i in range(5):
        temp_img.append(img_list[i])
        temp_msk.append(msk_list[i])
    #x += 1

imgs = np.array(temp_img)
msks = np.array(temp_msk)

#np.save("test_imgs", imgs)
#np.save("test_msks", msks)

In [ ]:
np.save("test_imgs", imgs)
np.save("test_msks", msks)